# Instrumentbræt for COVID-19 i Danmark

**Dette dokument viser forskellige nøgletal for COVID-19 i Danmark baseret på daglige tal fra Statens Serum Institut.**

*Koden til denne notebook findes på GitHub, se [github.com COVID-19-dk](https://github.com/covid-19-sim/covid-19-dk)*

## Datagrundlag

Data er indhentet fra Sundhedsstyrelsens hjemmeside for
[COVID-19 tal og overvågning](https://www.sst.dk/da/corona/tal-og-overvaagning).

Læs mere om datasættet på https://github.com/covid-19-sim/covid-19.sst.dk

In [ ]:
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import seaborn as sns
import scipy.optimize
from scipy.ndimage.filters import gaussian_filter1d

plt.rcParams['figure.figsize'] = [15, 10]
plt.rcParams['figure.facecolor'] = '0.9'

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/covid-19-sim/covid-19.sst.dk/master/total-covid-19-dk.csv',
                 parse_dates=['Date'])

# Kontroller at data har det forventede format
for c in ['Date', 'Tested', 'Confirmed', 'Total tested', 'Total confirmed', 'Hospitalised', 'ICU', 'ICU-vent', 'Deaths', 'Total deaths']:
    assert c in df.columns

In [ ]:
# Epidemistarten er den første dato
EPIDEMI_START = pd.Timestamp("2020-03-03")

df['Day'] = df['Date'] - EPIDEMI_START
df['Day N'] = df['Day'].dt.days

#
# Udvælg data vi interesserer os for
OVERSKRIFTER = {'Tested': 'Prøver',
                'Confirmed': 'Positive prøver',
                'Total tested': 'Prøver i alt',
                'Total confirmed': 'Positive prøver i alt',
                'Hospitalised': 'Indlagte',
                'ICU': 'Intensiv',
                'ICU-vent': 'Intensiv med respirator',
                'Total deaths': 'Døde'}

kolonner = list(OVERSKRIFTER.keys())

# Hvis ikke vi har observationen, så drop rækken
data = df[['Day N'] + kolonner]

In [ ]:
fig, axes = plt.subplots(nrows=2,ncols=int(np.ceil(len(kolonner)/2.)), sharey=True)
for i, kolonne in enumerate(kolonner):
    ax = axes.flat[i]
    ax.semilogy(data['Day N'], data[kolonne])
    ax.set_xlabel('Dag')
    ax.set_ylabel('Antal')
    ax.set_title(f'{OVERSKRIFTER[kolonne]}')
    ax.grid()

## Prøvetagning

Status på prøvetagningen kan vises på flere måder.

1. Første nøgletal er hvor mange prøver der er taget.
2. Andet nøgletal er forholdet mellem antal positive prøver i alt og total prøvetagning.
3. Tredie nøgletal er andelen af positive prøver på dagsbasis. Når prøvetagningen bliver bredere konvergerer det mod udbredelsen af sygdommen.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3)

ax = axes[0]
ax.plot(df['Day N'], df['Total tested'], marker='o')
ax.grid()
ax.set_title('Antal prøver i alt')
ax.set_xlabel('Dag')
ax.set_ylabel(OVERSKRIFTER['Total tested'])

ax = axes[1]
ax.loglog(df['Total tested'], df['Total confirmed'], marker='o')
ax.set_xlabel(OVERSKRIFTER['Total tested'])
ax.set_ylabel(OVERSKRIFTER['Total confirmed'])
ax.grid()
ax.set_title('Prøvetagning fremdrift')

ax = axes[2]
andel_positive = df['Confirmed']/df['Tested']
andel_positive_middel = andel_positive.rolling(7).mean()
ax.scatter(df['Day N'], andel_positive, marker='o', label='Andel positive')
ax.plot(df['Day N'], andel_positive_middel, '--', label='7-dages middel')
ax.grid()
ax.set_xlabel('Dag')
ax.set_ylabel('Andel positive')
ax.set_title('Andel positive prøver på dagsbasis')
ax.get_yaxis().set_major_formatter(mticker.PercentFormatter(xmax=1))
#handles, labels = ax.get_legend_handles_labels()
ax.legend(loc='upper right')
fig.tight_layout(pad=3)